### Imports

In [1]:
import nltk
import re
from collections import Counter
import string
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from nltk import bigrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from tqdm.notebook import tqdm
import numpy as np
import math
import statistics

[nltk_data] Downloading package stopwords to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sun Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Sun
[nltk_data]     Coliamco\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### import files

In [2]:
# import all the files in this block

data_frame_test = pd.read_csv("../../Data/Analysis_ready/clean_test_reddit.csv")
data_frame_odi = pd.read_csv(None)
data_frame_t20 = pd.read_csv(None)

In [3]:
data_frame_test.head()

,Post_id,Id,Time,Author,Score,No_comments,Body,Type,Pre-processed_body,Vader_Pre-processed_body
0,1f0syen,1f0syen,2024-08-25 20:04:21,NoQuestion4045,2445,281,Bangladesh beats Pakistan for the first ever i...,Post,bangladesh beat pakistan first ever test cricket,Bangladesh beats Pakistan for the first ever i...
1,1f0syen,lju4ld6,2024-08-25 20:07:28,deleted,1151,0,"Fort Rawalpindi, yet to be captured, by the ho...",Comment,fort rawalpindi yet capture home side,"Fort Rawalpindi , yet to be captured , by the ..."
2,1f0syen,lju9bzy,2024-08-25 20:58:53,mofucker20,289,0,That stat is so ridiculous. What’s even the us...,Comment,stat ridiculous whats even use home ground hav...,That stat is so ridiculous . What ’ s even the...
3,1f0syen,ljubwki,2024-08-25 21:24:17,SquiffyRae,173,0,Part of the problem is Pindi is such a highway...,Comment,part problem pindi highway road there's always...,Part of the problem is Pindi is such a highway...
4,1f0syen,ljvk09z,2024-08-26 02:17:08,deleted,48,0,It's particularly risky for a team like Pakist...,Comment,particularly risky team like pakistan tendency...,It's particularly risky for a team like Pakist...


# Graph: Interaction between formats for the most active members

### Identify the Top N most active members

In [4]:
# extract and sort comment count for non-deleted users

# test 
test_comment_count_by_author = data_frame_test.groupby('Author').size().reset_index(name='Comment/Post_Count')
test_comment_count_by_author = test_comment_count_by_author.sort_values(by='Comment/Post_Count', ascending=False)
test_comment_count_by_author = test_comment_count_by_author[test_comment_count_by_author['Author'] != "deleted"]

odi
odi_comment_count_by_author = data_frame_odi.groupby('Author').size().reset_index(name='Comment/Post_Count')
odi_comment_count_by_author = odi_comment_count_by_author.sort_values(by='Comment/Post_Count', ascending=False)
odi_comment_count_by_author = odi_comment_count_by_author[odi_comment_count_by_author['Author'] != "deleted"]

# t20
t20_comment_count_by_author = data_frame_t20.groupby('Author').size().reset_index(name='Comment/Post_Count')
t20_comment_count_by_author = t20_comment_count_by_author.sort_values(by='Comment/Post_Count', ascending=False)
t20_comment_count_by_author = t20_comment_count_by_author[t20_comment_count_by_author['Author'] != "deleted"]


In [5]:
head_count = 50

top_n_test_author_list = test_comment_count_by_author['Author'].head(head_count).to_list()
top_n_odi_author_list  = odi_comment_count_by_author['Author'].head(head_count).to_list()
top_n_t20_author_list  = t20_comment_count_by_author['Author'].head(head_count).to_list()

top_n_odi_author_list = []
top_n_t20_author_list = []


In [6]:
top_n_test_author_list[:5]

['VisRock',
 'abettertomorrow47',
 'Classymuch',
 'crashbandicoochy',
 'mongrelbifana']

### Filter the post and comments by the top users

In [7]:
# to prevent some users has barely drop off the top rank, I have included all top users in all top 3 group for filtering
top_n_users_set = set()
top_n_users_set.update(top_n_test_author_list)
top_n_users_set.update(top_n_odi_author_list)
top_n_users_set.update(top_n_t20_author_list)

to_n_author_content_test = data_frame_test[data_frame_test['Author'].isin(top_n_users_set)]
to_n_author_content_odi  = data_frame_odi[data_frame_odi['Author'].isin(top_n_users_set)]
to_n_author_content_t20  = data_frame_t20[data_frame_t20['Author'].isin(top_n_users_set)]

In [8]:
to_n_author_content_test.head()

,Post_id,Id,Time,Author,Score,No_comments,Body,Type,Pre-processed_body,Vader_Pre-processed_body
36,1f0syen,ljuphhc,2024-08-25 23:15:35,confused_brown_dude,5,0,I wasn’t ready for this 😭,Comment,wasnt ready,I wasn ’ t ready for this 😭
65,1f0syen,ljuppbl,2024-08-25 23:17:07,confused_brown_dude,30,0,"I can’t get past the idiot thing 🤣, why does i...",Comment,cant get past idiot thing hit hard,"I can ’ t get past the idiot thing 🤣 , why doe..."
72,1f0syen,ljuv8bq,2024-08-25 23:53:59,Medical_Turing_Test,12,0,As a Fawad Alam truther it's important to note...,Comment,fawad alam truther important note ss sensational,As a Fawad Alam truther it's important to note...
74,1f0syen,ljv3br3,2024-08-26 00:42:41,Medical_Turing_Test,10,0,Probably. But Fawad is ancient.\n\nUnlikely. S...,Comment,probably fawad ancient unlikely saim ayub scor...,Probably . But Fawad is ancient . Unlikely . S...
76,1f0syen,ljv8d1c,2024-08-26 01:11:04,Medical_Turing_Test,4,0,https://www.espncricinfo.com/series/quaid-e-az...,Comment,please stop embarrassing,Please stop embarrassing yourself


In [9]:
to_n_author_content_test.shape

(15105, 10)

In [13]:
# TODO: Remember to import networkx
import networkx as nx

# here are the three main format
test = "Test"
odi  = "ODI"
t20  = "T20"

activity_graph = nx.DiGraph()
activity_graph.add_node(test)
activity_graph.add_node(odi)
activity_graph.add_node(t20)

user_check_set = set()

# to_n_author_content_test
# acc = author comment count
# test:
acc_test_list = to_n_author_content_test.groupby('Author').size().reset_index(name='Comment/Post_Count')

for index, row in acc_test_list.iterrows():
    
    user = row['Author']
    if user not in user_check_set:
        user_check_set.add(user)
        activity_graph.add_node(user)
    
    activity_graph.add_edge(user, test, weight=row['Comment/Post_Count'])

# ODI
acc_odi_list = to_n_author_content_odi.groupby('Author').size().reset_index(name='Comment/Post_Count')

for index, row in acc_odi_list.iterrows():
    
    user = row['Author']
    if user not in user_check_set:
        user_check_set.add(user)
        activity_graph.add_node(user)
    
    activity_graph.add_edge(user, odi, weight=row['Comment/Post_Count'])

# ODI
acc_t20_list = to_n_author_content_t20.groupby('Author').size().reset_index(name='Comment/Post_Count')

for index, row in acc_t20_list.iterrows():
    
    user = row['Author']
    if user not in user_check_set:
        user_check_set.add(user)
        activity_graph.add_node(user)
    
    activity_graph.add_edge(user, t20, weight=row['Comment/Post_Count'])


In [15]:
nx.readwrite.write_graphml(activity_graph, 'test_graph.graphml')